# 1. 简单的QUBO模型
先试一试[这篇文章里的QUBO模型](https://ieeexplore.ieee.org/document/9367437)

我们用一个简单的例子。只有3个节点的完全图，边权重分别为[1, 2, 3]

**变量：**<br>
$x_i$: For each edge

**目标函数：**最小化选取的边的权重之和

**约束：**总共选取$V-1$条边

In [1]:
from sympy import symbols, expand, collect

# 定义三个符号变量
x1, x2, x3 = symbols('x1 x2 x3')

# 定义权重
w1, w2, w3 = symbols('w1 w2 w3')

# 定义惩罚系数
P = symbols('P')

# 目标函数的线性和二次项
objective = w1*x1 + w2*x2 + w3*x3

# 约束条件：x1 + x2 + x3 = 2
# 我们通过添加一个平方项将它转换成二次形式
# 并将其作为惩罚项添加到目标函数中
constraint = P*(x1 + x2 + x3 - 2)**2

# 合并目标函数和约束项
expanded_objective = expand(objective + constraint)

# 展示展开后的表达式
expanded_objective


P*x1**2 + 2*P*x1*x2 + 2*P*x1*x3 - 4*P*x1 + P*x2**2 + 2*P*x2*x3 - 4*P*x2 + P*x3**2 - 4*P*x3 + 4*P + w1*x1 + w2*x2 + w3*x3

In [2]:
# 因为在QUBO问题中 x_i^2 = x_i（因为x_i是二进制变量），所以我们替换掉所有的x_i^2项
simplified_objective = expanded_objective.subs({x1**2: x1, x2**2: x2, x3**2: x3})

simplified_objective

2*P*x1*x2 + 2*P*x1*x3 - 3*P*x1 + 2*P*x2*x3 - 3*P*x2 - 3*P*x3 + 4*P + w1*x1 + w2*x2 + w3*x3

In [3]:

# 定义新的符号变量表示乘积项
x1x2, x1x3, x2x3 = symbols('x1x2 x1x3 x2x3')

# 替换原表达式中的乘积项为新的符号变量
replaced_objective = simplified_objective.subs({x1*x2: x1x2, x1*x3: x1x3, x2*x3: x2x3})

# 合并同类项，包括新定义的乘积项符号变量
final_objective_with_products = collect(replaced_objective, [x1, x2, x3, x1x2, x1x3, x2x3])

final_objective_with_products

2*P*x1x2 + 2*P*x1x3 + 2*P*x2x3 + 4*P + x1*(-3*P + w1) + x2*(-3*P + w2) + x3*(-3*P + w3)

In [12]:

w = [10, 2, 3]
# 根据上面整理得到的公式填写QUBO矩阵
P = max(w)*len(w)
Q = {(i, j): 0 for i in range(3) for j in range(3)}
for i in range(3):
    Q[(i, i)] += -3*P + w[i]

Q[(0, 1)] += 2*P
Q[(1, 0)] += 2*P

Q[(0, 2)] += 2*P
Q[(2, 0)] += 2*P

Q[(1, 2)] += 2*P
Q[(2, 1)] += 2*P

Q

{(0, 0): -80,
 (0, 1): 60,
 (0, 2): 60,
 (1, 0): 60,
 (1, 1): -88,
 (1, 2): 60,
 (2, 0): 60,
 (2, 1): 60,
 (2, 2): -87}

In [13]:
# 求解
from dwave.system import DWaveSampler, EmbeddingComposite
dwave_sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
sampleset = dwave_sampler.sample_qubo(Q, num_reads = 1000)
print(sampleset)

   0  1  2 energy num_oc. chain_.
0  0  1  0  -88.0     470     0.0
1  0  0  1  -87.0     410     0.0
2  1  0  0  -80.0     120     0.0
['BINARY', 3 rows, 1000 samples, 3 variables]


## 1.1 把上述QUBO求解过程整合到该格单独运行即可
由于量子退火器的随机性，我测试了很多次都只有3个invalid solutions（i.e. 3个点的图里只选择了1条边，但是我们需要的是2条边的解），所以写了一个while loop来自动运行，直到出现第四个解。由于该例子的完全图是3条边，前三种解大概率会挨个把每条边选一次，所以第四个解一定包含了至少2条边。

In [25]:
from dwave.system import DWaveSampler, EmbeddingComposite
dwave_sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
w = [10, 2, 3] # 边权重
sampleset = []
k = 0
while (len(sampleset) <= 3) and (k < 100):
    # 根据上面整理得到的公式填写QUBO矩阵
    P = max(w)*len(w)
    Q = {(i, j): 0 for i in range(3) for j in range(3)}
    for i in range(3):
        Q[(i, i)] += -3*P + w[i]
    
    Q[(0, 1)] += 2*P
    Q[(1, 0)] += 2*P
    
    Q[(0, 2)] += 2*P
    Q[(2, 0)] += 2*P
    
    Q[(1, 2)] += 2*P
    Q[(2, 1)] += 2*P
    sampleset = dwave_sampler.sample_qubo(Q, num_reads = 1000)
    k += 1
print(sampleset)

   0  1  2 energy num_oc. chain_.
0  0  1  0  -88.0     618     0.0
1  0  0  1  -87.0     224     0.0
2  1  0  0  -80.0     156     0.0
3  1  1  0  -48.0       1     0.0
4  1  0  1  -47.0       1     0.0
['BINARY', 5 rows, 1000 samples, 3 variables]


## 1.2 自动展开与整理公式
这个函数可以作为建立QUBO矩阵的参考

In [7]:
def create_final_objective_with_products(num_variables, penalty, constraint_value):
    # 创建符号变量和权重
    x = symbols(f'x1:{num_variables+1}')
    w = symbols(f'w1:{num_variables+1}')
    
    # 构建目标函数
    objective = sum(w[i] * x[i] for i in range(num_variables))
    
    # 构建约束条件
    constraint = penalty * (sum(x) - constraint_value)**2
    
    # 合并目标函数和约束项
    expanded_objective = expand(objective + constraint)

    # 替换每个变量的平方项
    replacements = {x[i]**2: x[i] for i in range(num_variables)}
    simplified_objective = expanded_objective.subs(replacements)

    # 为所有变量对生成唯一的乘积符号
    product_symbols = symbols(' '.join(f'x{i+1}x{j+1}' for i in range(num_variables) for j in range(i+1, num_variables)))
    product_replacements = {x[i] * x[j]: product_symbols[i * (num_variables - 1) - (i * (i + 1)) // 2 + j - i - 1]
                            for i in range(num_variables) for j in range(i+1, num_variables)}
    replaced_objective = simplified_objective.subs(product_replacements)

    # 合并同类项，包括新定义的乘积项符号变量
    final_objective = collect(replaced_objective, [x[i] for i in range(num_variables)] + list(product_replacements.values()))

    return final_objective

# 示例：对于4条边的情况
num_variables = 4
penalty = symbols('P')
constraint_value = 2

final_objective_example = create_final_objective_with_products(num_variables, penalty, constraint_value)
final_objective_example



2*P*x1x2 + 2*P*x1x3 + 4*P*x1x4 + 4*P*x2x3 + 4*P + x1*(-3*P + w1) + x2*(-3*P + w2) + x3*(-3*P + w3) + x4*(-3*P + w4)

## 1.3 简单QUBO模型的general形式

初始图

In [69]:
vertices = ["A", "B", "C"]
edges = [("A", "B"), ("A", "C"), ("B", "C")]
weights = {("A", "B"): 10, ("A", "C"): 2, ("B", "C"): 3}
V = len(vertices)
E = len(edges)

给每条边加变量

In [70]:
variable_index = {edge: i for i, edge in enumerate(edges)}
variable_index

{('A', 'B'): 0, ('A', 'C'): 1, ('B', 'C'): 2}

初始化QUBO矩阵与建模

In [71]:
lambda_A = max(weights.values()) * E
# Initialize QUBO dictionary with all possible pairs of variable indices
Q = {(variable_index[key1], variable_index[key2]): 0 for key1 in variable_index for key2 in variable_index}

for i in range(E):
    edge = edges[i]
    xi = variable_index[edge]
    # 目标函数
    Q[(xi, xi)] += weights[edge]
    Q[(xi, xi)] += (3 - 2*V)*lambda_A
    for j in range(i+1, E):
        xj = variable_index[edges[j]]
        # 约束
        Q[(xi, xj)] += 2*lambda_A
        Q[(xj, xi)] += 2*lambda_A # 对称

Q

{(0, 0): -80,
 (0, 1): 60,
 (0, 2): 60,
 (1, 0): 60,
 (1, 1): -88,
 (1, 2): 60,
 (2, 0): 60,
 (2, 1): 60,
 (2, 2): -87}

求解

**不要开VPN**

### Quantum Sampler:

In [72]:
from dwave.system import DWaveSampler, EmbeddingComposite
dwave_sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
sampleset = dwave_sampler.sample_qubo(Q, num_reads = 1000)
print(sampleset)

SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project DEV

In [73]:
import dimod
# Now you would typically use a QUBO solver to find the minimum of this matrix
# For this example, we will use a classical simulated annealing sampler
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample_qubo(Q, num_reads=100)
print(response)

    0  1  2 energy num_oc.
0   0  1  0  -88.0       1
1   0  1  0  -88.0       1
7   0  1  0  -88.0       1
8   0  1  0  -88.0       1
10  0  1  0  -88.0       1
13  0  1  0  -88.0       1
17  0  1  0  -88.0       1
18  0  1  0  -88.0       1
19  0  1  0  -88.0       1
20  0  1  0  -88.0       1
22  0  1  0  -88.0       1
23  0  1  0  -88.0       1
26  0  1  0  -88.0       1
29  0  1  0  -88.0       1
31  0  1  0  -88.0       1
37  0  1  0  -88.0       1
41  0  1  0  -88.0       1
42  0  1  0  -88.0       1
43  0  1  0  -88.0       1
45  0  1  0  -88.0       1
47  0  1  0  -88.0       1
49  0  1  0  -88.0       1
55  0  1  0  -88.0       1
58  0  1  0  -88.0       1
60  0  1  0  -88.0       1
63  0  1  0  -88.0       1
64  0  1  0  -88.0       1
66  0  1  0  -88.0       1
67  0  1  0  -88.0       1
71  0  1  0  -88.0       1
74  0  1  0  -88.0       1
76  0  1  0  -88.0       1
79  0  1  0  -88.0       1
81  0  1  0  -88.0       1
83  0  1  0  -88.0       1
87  0  1  0  -88.0       1
9

## 1.4 使用Klein data聚类后簇之间的连接矩阵

In [58]:
# 初始化图。现在先手打，后续可以导入scRNA RAW data处理到这一步
vertices = ["ES", "2-days", "4-days", "7-days"]
edges = [("ES", "2-days"), ("ES", "4-days"), ("2-days", "4-days"), ("4-days", "7-days")]
weights = {("ES", "2-days"):1.76650, ("ES", "4-days"):3.029646, 
           ("2-days", "4-days"):1.885074, ("4-days", "7-days"):1.831101}
V = len(vertices)
E = len(edges)

# 每条边的变量
variable_index = {edge: i for i, edge in enumerate(edges)}
print(variable_index)

# 初始化QUBO矩阵并建模
lambda_A = max(weights.values()) * E
# Initialize QUBO dictionary with all possible pairs of variable indices
Q = {(variable_index[key1], variable_index[key2]): 0 for key1 in variable_index for key2 in variable_index}

for i in range(E):
    edge = edges[i]
    xi = variable_index[edge]
    # 目标函数
    Q[(xi, xi)] += weights[edge]
    Q[(xi, xi)] += (1-2*(V-1))*lambda_A
    for j in range(i+1, E):
        xj = variable_index[edges[j]]
        # 约束
        Q[(xi, xj)] += 2*lambda_A
        Q[(xj, xi)] += 2*lambda_A # 对称

{('ES', '2-days'): 0, ('ES', '4-days'): 1, ('2-days', '4-days'): 2, ('4-days', '7-days'): 3}


In [67]:
# 求解
# from dwave.system import DWaveSampler, EmbeddingComposite
# dwave_sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
# sampleset = dwave_sampler.sample_qubo(Q, num_reads = 1000)
# print(sampleset)

In [68]:
import dimod
# Now you would typically use a QUBO solver to find the minimum of this matrix
# For this example, we will use a classical simulated annealing sampler
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample_qubo(Q, num_reads=100)
print(response)

    0  1  2  3     energy num_oc.
6   1  0  0  1 -69.113903       1
8   1  0  0  1 -69.113903       1
10  1  0  0  1 -69.113903       1
17  1  0  0  1 -69.113903       1
21  1  0  0  1 -69.113903       1
25  1  0  0  1 -69.113903       1
30  1  0  0  1 -69.113903       1
33  1  0  0  1 -69.113903       1
35  1  0  0  1 -69.113903       1
53  1  0  0  1 -69.113903       1
59  1  0  0  1 -69.113903       1
62  1  0  0  1 -69.113903       1
65  1  0  0  1 -69.113903       1
73  1  0  0  1 -69.113903       1
74  1  0  0  1 -69.113903       1
77  1  0  0  1 -69.113903       1
87  1  0  0  1 -69.113903       1
91  1  0  0  1 -69.113903       1
94  1  0  0  1 -69.113903       1
0   1  0  1  0  -69.05993       1
3   1  0  1  0  -69.05993       1
7   1  0  1  0  -69.05993       1
9   1  0  1  0  -69.05993       1
12  1  0  1  0  -69.05993       1
15  1  0  1  0  -69.05993       1
19  1  0  1  0  -69.05993       1
26  1  0  1  0  -69.05993       1
27  1  0  1  0  -69.05993       1
34  1  0  1  0

### 一些调取结果信息的D wave指令
[SampleSet Object's man page](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampleset.html)

In [45]:
sampleset.info

{'timing': {'qpu_sampling_time': 73580.0,
  'qpu_anneal_time_per_sample': 20.0,
  'qpu_readout_time_per_sample': 32.56,
  'qpu_access_time': 80425.61,
  'qpu_access_overhead_time': 537.39,
  'qpu_programming_time': 6845.61,
  'qpu_delay_time_per_sample': 21.02,
  'total_post_processing_time': 479.0,
  'post_processing_overhead_time': 479.0},
 'problem_id': '40edd12b-bdc4-48c5-9c3f-6acf9b9da6bc'}

In [46]:
sampleset.variables

Variables([0, 1, 2, 3])

In [47]:
sampleset.record

rec.array([([1, 0, 0, 1], -69.113903, 242, 0.),
           ([1, 0, 1, 0], -69.05993 , 275, 0.),
           ([0, 0, 1, 1], -68.995329, 170, 0.),
           ([1, 1, 0, 0], -67.915358, 129, 0.),
           ([0, 1, 0, 1], -67.850757,  80, 0.),
           ([0, 1, 1, 0], -67.796784, 104, 0.)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')])

In [48]:
sampleset.vartype

<Vartype.BINARY: frozenset({0, 1})>

# 2. 基于k-MST的复杂的QUBO模型

初始化图

In [195]:
import itertools
k = 3
vertices = ["A", "B", "C"]
edges = [("A", "B"), ("A", "C"), ("B", "C")]
weights = {("A", "B"): 1, ("A", "C"): 2, ("B", "C"): 3}

建立$x_{u, i}$的唯一索引

In [196]:
# Create a binary variable index for vertex positions in the permutation and for the connections between vertices
# x[v, i] - vertex v is in the i-th position in the permutation
# y[v, i] - vertex v is connected to the vertex at the i-th position
variable_index = {var: idx for idx, var in enumerate(itertools.product(vertices, range(k)))}
variable_index # x_{u, i}

{('A', 0): 0,
 ('A', 1): 1,
 ('A', 2): 2,
 ('B', 0): 3,
 ('B', 1): 4,
 ('B', 2): 5,
 ('C', 0): 6,
 ('C', 1): 7,
 ('C', 2): 8}

后续要继续建立$y$变量，所以了解一下总共目前有多少$x$变量了，接下去


有$|V|^2$个$x_{u, i}$变量

In [197]:
y_index_offset = len(variable_index)  # Offset for y variables
y_index_offset

9

建立$y_{v, i}$变量的唯一索引

**总共有$2|V|k - |V|$个变量**

In [198]:
# Add indices for y variables
for v in vertices:
    for i in range(k-1):
        variable_index[(v, i, 'y')] = y_index_offset + (k-1) * vertices.index(v) + i
variable_index

{('A', 0): 0,
 ('A', 1): 1,
 ('A', 2): 2,
 ('B', 0): 3,
 ('B', 1): 4,
 ('B', 2): 5,
 ('C', 0): 6,
 ('C', 1): 7,
 ('C', 2): 8,
 ('A', 0, 'y'): 9,
 ('A', 1, 'y'): 10,
 ('B', 0, 'y'): 11,
 ('B', 1, 'y'): 12,
 ('C', 0, 'y'): 13,
 ('C', 1, 'y'): 14}

生成所有的vertex pairs

In [199]:
import itertools
# Generate all pairs (including pairs with the same elements)
all_pairs = list(itertools.product(vertices, repeat=2))
# all_pairs

In [200]:
# Initialize QUBO dictionary with all possible pairs of variable indices
Q = {(variable_index[key1], variable_index[key2]): 0 for key1 in variable_index for key2 in variable_index}

# Now Q is a dictionary with keys representing all possible pairs of indices and values all set to 0
Q

{(0, 0): 0,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (0, 4): 0,
 (0, 5): 0,
 (0, 6): 0,
 (0, 7): 0,
 (0, 8): 0,
 (0, 9): 0,
 (0, 10): 0,
 (0, 11): 0,
 (0, 12): 0,
 (0, 13): 0,
 (0, 14): 0,
 (1, 0): 0,
 (1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (1, 4): 0,
 (1, 5): 0,
 (1, 6): 0,
 (1, 7): 0,
 (1, 8): 0,
 (1, 9): 0,
 (1, 10): 0,
 (1, 11): 0,
 (1, 12): 0,
 (1, 13): 0,
 (1, 14): 0,
 (2, 0): 0,
 (2, 1): 0,
 (2, 2): 0,
 (2, 3): 0,
 (2, 4): 0,
 (2, 5): 0,
 (2, 6): 0,
 (2, 7): 0,
 (2, 8): 0,
 (2, 9): 0,
 (2, 10): 0,
 (2, 11): 0,
 (2, 12): 0,
 (2, 13): 0,
 (2, 14): 0,
 (3, 0): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0,
 (3, 4): 0,
 (3, 5): 0,
 (3, 6): 0,
 (3, 7): 0,
 (3, 8): 0,
 (3, 9): 0,
 (3, 10): 0,
 (3, 11): 0,
 (3, 12): 0,
 (3, 13): 0,
 (3, 14): 0,
 (4, 0): 0,
 (4, 1): 0,
 (4, 2): 0,
 (4, 3): 0,
 (4, 4): 0,
 (4, 5): 0,
 (4, 6): 0,
 (4, 7): 0,
 (4, 8): 0,
 (4, 9): 0,
 (4, 10): 0,
 (4, 11): 0,
 (4, 12): 0,
 (4, 13): 0,
 (4, 14): 0,
 (5, 0): 0,
 (5, 1): 0,
 (5, 2): 0,
 (5, 3): 0,
 (5, 4): 0,
 (5, 5): 0,
 (5

编写目标函数和约束进QUBO矩阵

In [201]:
lambda_A = 2 # 惩罚系数λ_A
lambda_B = -1 # 奖励系数λ_B
# Test if an edge (u, v) is present in the tree
# HTree
for i in range(k-1):
    for u, v in all_pairs:
            if u != v and (u, v) in edges: # HTree第一项
                # Add a reward if an edge (u, v) is present in the tree
                Q[(variable_index[(u, i)], variable_index[(v, i, 'y')])] += lambda_B * weights[(u, v)]
                Q[(variable_index[(v, i, 'y')], variable_index[(u, i)])] += lambda_B * weights[(u, v)] # Symmetric
            else: # HTree第二项
                Q[(variable_index[(u, i)], variable_index[(v, i, 'y')])] += lambda_A
                Q[(variable_index[(v, i, 'y')], variable_index[(u, i)])] += lambda_A # Symmetric



# H(1)Cst第一项
for p in range(k):
    v = vertices[p]
    for q in range(p+1, k):
        u = vertices[q]
        for i in range(k):
            Q[(variable_index[(u, i)], variable_index[(v, i)])] += lambda_A
            Q[(variable_index[(v, i)], variable_index[(u, i)])] += lambda_A # Symmetric

# H(1)Cst第二项
for i in range(k):
    for j in range(i + 1, k):
        for v in vertices:
            Q[(variable_index[(v, i)], variable_index[(v, j)])] += lambda_A
            Q[(variable_index[(v, j)], variable_index[(v, i)])] += lambda_A # Symmetric

# H(1)Cst第三项
for i in range(k):
    for v_idx in range(k):
        v = vertices[v_idx]
        xv = variable_index[(v, i)]
        Q[(xv, xv)] += -lambda_A
        for u_idx in range(v_idx+1, k):
            u = vertices[u_idx]
            xu = variable_index[(u, i)]
            Q[(xu, xv)] += 2*lambda_A
            Q[(xv, xu)] += 2*lambda_A # Symmetric

# H(2)Cst         
for v in vertices:
    # Add constraint 1: A vertex only connects to previous vertices in the sequence
    # H(2)Cst的第一项
    for i in range(1, k):  # Start from 1 because the first position cannot have a previous vertex
        for u in vertices:
            if u != v:
                # Add a penalty if a vertex tries to connect to a vertex ahead of it in the sequence
                Q[(variable_index[(v, i)], variable_index[(u, i-1, 'y')])] += lambda_A
                Q[(variable_index[(u, i-1, 'y')], variable_index[(v, i)])] += lambda_A # Symmetric
    
    # H(2)Cst的第二项
    for i in range(2, k):
        xi = variable_index[(v, i)]        
        yi = variable_index[(v, i-1, 'y')]
        # Square of the difference between x[v, i] and y[v, i-1]
        Q[(xi, xi)] += lambda_A
        Q[(yi, yi)] += lambda_A
        Q[(xi, yi)] += - 2 * lambda_A
        Q[(yi, xi)] += - 2 * lambda_A # Symemetric
        for j in range(i+1, k):
            xj = variable_index[(v, j)]
            yj = variable_index[(v, j - 1, 'y')]
            Q[(xi, xj)] += 2*lambda_A
            Q[(xj, xi)] += 2*lambda_A # Symmetric
            
            Q[(yi, yj)] += 2*lambda_A
            Q[(yj, yi)] += 2*lambda_A # Symmetric

Q  # Display the QUBO dictionary with the constraints added

{(0, 0): -2,
 (0, 1): 2,
 (0, 2): 2,
 (0, 3): 6,
 (0, 4): 0,
 (0, 5): 0,
 (0, 6): 6,
 (0, 7): 0,
 (0, 8): 0,
 (0, 9): 2,
 (0, 10): 0,
 (0, 11): -1,
 (0, 12): 0,
 (0, 13): -2,
 (0, 14): 0,
 (1, 0): 2,
 (1, 1): -2,
 (1, 2): 2,
 (1, 3): 0,
 (1, 4): 6,
 (1, 5): 0,
 (1, 6): 0,
 (1, 7): 6,
 (1, 8): 0,
 (1, 9): 0,
 (1, 10): 2,
 (1, 11): 2,
 (1, 12): -1,
 (1, 13): 2,
 (1, 14): -2,
 (2, 0): 2,
 (2, 1): 2,
 (2, 2): 0,
 (2, 3): 0,
 (2, 4): 0,
 (2, 5): 6,
 (2, 6): 0,
 (2, 7): 0,
 (2, 8): 6,
 (2, 9): 0,
 (2, 10): -4,
 (2, 11): 0,
 (2, 12): 2,
 (2, 13): 0,
 (2, 14): 2,
 (3, 0): 6,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): -2,
 (3, 4): 2,
 (3, 5): 2,
 (3, 6): 6,
 (3, 7): 0,
 (3, 8): 0,
 (3, 9): 2,
 (3, 10): 0,
 (3, 11): 2,
 (3, 12): 0,
 (3, 13): -3,
 (3, 14): 0,
 (4, 0): 0,
 (4, 1): 6,
 (4, 2): 0,
 (4, 3): 2,
 (4, 4): -2,
 (4, 5): 2,
 (4, 6): 0,
 (4, 7): 6,
 (4, 8): 0,
 (4, 9): 2,
 (4, 10): 2,
 (4, 11): 0,
 (4, 12): 2,
 (4, 13): 2,
 (4, 14): -3,
 (5, 0): 0,
 (5, 1): 0,
 (5, 2): 6,
 (5, 3): 2,
 (5, 4): 2,
 (5,

求解

In [202]:
import dimod
# Now you would typically use a QUBO solver to find the minimum of this matrix
# For this example, we will use a classical simulated annealing sampler
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample_qubo(Q, num_reads=10)
print(response)

# # The response will contain the solution(s) to the QUBO problem
# for sample, energy in response.data(['sample', 'energy']):
#     print('Sample: ', sample, 'Energy: ', energy)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 energy num_oc.
1  1  0  0  0  1  0  0  0  1  0  0  1  0  0  1  -18.0       1
4  1  0  0  0  1  0  0  0  1  0  0  1  0  1  1  -18.0       1
5  1  0  0  0  1  0  0  0  1  0  0  1  0  0  1  -18.0       1
7  1  0  0  0  1  0  0  0  1  0  0  1  0  1  1  -18.0       1
8  1  0  0  0  1  0  0  0  1  0  0  1  0  0  1  -18.0       1
9  1  0  0  0  1  0  0  0  1  0  0  1  0  1  1  -18.0       1
0  0  1  0  1  0  0  0  0  1  0  0  0  0  1  1  -16.0       1
2  1  0  0  0  0  1  0  0  0  0  0  1  1  1  0  -14.0       1
3  0  0  1  1  0  0  0  0  0  0  1  0  0  1  0  -14.0       1
6  0  0  1  1  0  0  0  0  0  0  1  0  0  1  0  -14.0       1
['BINARY', 10 rows, 10 samples, 15 variables]
